In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
import cv2
import imageio
import matplotlib.pyplot as plt
import io
from tqdm.auto import tqdm

In [2]:
frame_folders = [f for f in Path('./frames').glob("*") if f.is_dir() and '1025B' in f.name]
output_folder = Path('./output')
visualize_folder = Path('./visualize')

In [3]:
def merge_file(cut_imgs,df1,df2):
    ids = [img.name for img in cut_imgs]
    df = pd.DataFrame(data={'frame':ids})
    df = pd.merge(df,df1,on='frame',how='left')
    df = pd.merge(df,df2,on='frame',how='left')
    return df
past_value=np.nan

def fill_nan(value):
    global past_value
    if pd.isna(value):
        return past_value
    else:
        past_value = value
        return value

def merge_mask(df,cut_imgs):
    frame = ["_".join(img.name.split("_")[:-1])+".png" for img in cut_imgs]
    masks = [img.name for img in cut_imgs]
    df1 = pd.DataFrame(data={'frame':frame,'masks':masks})
    df = pd.merge(df,df1,on='frame',how='left')
    return df

In [4]:
frame_folder = frame_folders[1]
vid = frame_folders[1].name
frame_imgs = sorted(list(frame_folders[1].glob("*.png")))
cut_imgs = sorted(list((output_folder/vid).glob("*.png")))
anno_file = output_folder/f'{vid}_anno.pkl'
view_file = output_folder/f'{vid}_result.csv'
select_file = output_folder/f'{vid}_selection.csv'
df_view = pd.read_csv(view_file)
df_select = pd.read_csv(select_file)

In [5]:
df = merge_file(frame_imgs,df_view,df_select)
df = merge_mask(df,cut_imgs)
past_value=np.nan
df['viewport'] = df.viewport.apply(fill_nan)
past_value=np.nan
df['selection'] = df.selection.apply(fill_nan)
past_value=np.nan
df['masks'] = df.masks.apply(fill_nan)

In [6]:
df.head()

,frame,viewport,selection,masks
0,1025B_Avatar_Workroom_cut_0001.png,NaN,NaN,NaN
1,1025B_Avatar_Workroom_cut_0002.png,NaN,NaN,NaN
2,1025B_Avatar_Workroom_cut_0003.png,NaN,NaN,NaN
3,1025B_Avatar_Workroom_cut_0004.png,NaN,NaN,NaN
4,1025B_Avatar_Workroom_cut_0005.png,NaN,NaN,NaN


In [7]:
img_list=[]
for index,line in df.iterrows():
    img_path = frame_folder/line['frame']
    img = cv2.imread(str(img_path))[:,:,::-1]
    img_list.append(img)
imageio.v2.mimwrite(visualize_folder/f'{vid}_org.mp4',img_list,fps=10)

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (720, 405) to (720, 416) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


In [8]:
img_list=[]
x=['menu','avatar','detail']
y=[0,0,0]
for index,line in tqdm(df.iterrows()):
    data = line['viewport']
    if data=='menu':
        y[0]+=1
    elif data=='avatar':
        y[1]+=1
    elif data=='detail':
        y[2]+=1
    y_max = max(max(y),1)
    fig,ax = plt.subplots()
    ax.bar(x,y,width=1,edgecolor='white',linewidth=0.7)
    ax.set(ylim=(0,y_max),yticks=np.arange(1,y_max+1,min(10,y_max+1)))
    io_buf = io.BytesIO()
    fig.savefig(io_buf, format='raw')
    io_buf.seek(0)
    img_arr = np.reshape(np.frombuffer(io_buf.getvalue(), dtype=np.uint8),
                         newshape=(int(fig.bbox.bounds[3]), int(fig.bbox.bounds[2]), -1))
    io_buf.close()
    img_list.append(img_arr)
    plt.close()
imageio.v2.mimwrite(visualize_folder/f'{vid}_menu.mp4',img_list,fps=10)

0it [00:00, ?it/s]

In [9]:
img_list=[]
labels=[]
y = []
for index,line in tqdm(df.iterrows()):
    data = line['selection']

    if data not in labels:
        labels.append(data)
        y.append(1)
    else:
        y[labels.index(data)] +=1
    fig,ax = plt.subplots()
    ax.pie(y,labels=labels, autopct='%1.1f%%')
    io_buf = io.BytesIO()
    fig.savefig(io_buf, format='raw')
    io_buf.seek(0)
    img_arr = np.reshape(np.frombuffer(io_buf.getvalue(), dtype=np.uint8),
                         newshape=(int(fig.bbox.bounds[3]), int(fig.bbox.bounds[2]), -1))
    io_buf.close()
    img_list.append(img_arr)
    plt.close()
imageio.v2.mimwrite(visualize_folder/f'{vid}_selection.mp4',img_list,fps=10)

0it [00:00, ?it/s]

In [10]:
img_list=[]
for index,line in tqdm(df.iterrows()):
    if pd.isna(line['masks']):
        img_list.append(np.zeros([220,80,3],dtype=np.uint8))
    else:
        img_path = (output_folder/vid)/line['masks']
        img = cv2.imread(str(img_path))
        img = cv2.resize(img,(80,220))
        img_list.append(img[:,:,::-1])
imageio.v2.mimwrite(visualize_folder/f'{vid}_masks.mp4',img_list,fps=10)

0it [00:00, ?it/s]

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (80, 220) to (80, 224) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


In [12]:
img_list=[]
fid='1025B_2'
f_list = sorted(list((visualize_folder/f"{fid}").glob("*.png")))
for file in f_list:
    img = cv2.imread(str(file))[:,:,::-1]
    img_list.append(img)
imageio.v2.mimwrite(visualize_folder/f'{fid}_org.mp4',img_list,fps=10)

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (1920, 1080) to (1920, 1088) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
